# Regressions in physics with scikit-learn

In this notebook we will look at some posibilities for fitting data and reproducing the data-generating procedure.

## Create synthetic polynomial data

In [ ]:
import numpy as np

rng = np.random.RandomState(0)

n_sample = 100
data_max, data_min = 1.4, -1.4
len_data = (data_max - data_min)
# sort the data to make plotting easier later
data = np.sort(rng.rand(n_sample) * len_data - len_data / 2)
noise = rng.randn(n_sample) * .5
target = data ** 3 - 0.5 * data ** 2 + noise

In [ ]:
import pandas as pd

full_data = pd.DataFrame({"input_feature": data, "target": target})
true_coefs = pd.DataFrame({'intercept': [0.], 'weight_X': [0.],
                           'weight_X^2':[-0.5], 'weight_X^3': [1.0]})

In [ ]:
import seaborn as sns

_ = sns.scatterplot(data=full_data, x="input_feature", y="target",
                    color="black", alpha=0.5)

In scikit-learn, by convention data (also called X in the scikit-learn documentation) should be a 2D matrix of shape (n_samples, n_features). If data is a 1D vector, you need to reshape it into a matrix with a single column if the vector represents a feature or a single row if the vector represents a sample.

In [ ]:
# X should be 2D for sklearn: (n_samples, n_features)
data = data.reshape((-1, 1))
data.shape

## Try different regressions

A common first step would be to try the simplest option: fitting a straight line as a baseline.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

linear_regression = LinearRegression()
linear_regression.fit(data, target)
target_predicted = linear_regression.predict(data)

In [ ]:
mse = mean_squared_error(target, target_predicted)

In [ ]:
ax = sns.scatterplot(data=full_data, x="input_feature", y="target",
                     color="black", alpha=0.5)
ax.plot(data, target_predicted)
_ = ax.set_title(f"Mean squared error = {mse:.2f}")

In [ ]:
print(f"weight: {linear_regression.coef_[0]:.2f}, "
      f"intercept: {linear_regression.intercept_:.2f}")

It is important to note that the learnt model will not be able to handle the non-linear relationship between data and target since linear models assume the relationship between data and target to be linear.

Indeed, there are 3 possibilities to solve this issue:

    - choose a model that can natively deal with non-linearity,

    - engineer a richer set of features by including expert knowledge which can be directly used by a simple linear model, or

    - use a “kernel” to have a locally-based decision function instead of a global linear decision function.

Let’s illustrate quickly the first point by using a decision tree regressor which can natively handle non-linearity.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor(max_depth=3).fit(data, target)
target_predicted = tree.predict(data)
mse = mean_squared_error(target, target_predicted)

In [ ]:
ax = sns.scatterplot(data=full_data, x="input_feature", y="target",
                     color="black", alpha=0.5)
ax.plot(data, target_predicted)
_ = ax.set_title(f"Mean squared error = {mse:.2f}")

The problem with this approach is interpretability of the model in terms of first principles. The same applies to using a kernel.

Instead we could also modify our data: we could create new features, derived from the original features, using some expert knowledge. In this example, we know that we have a cubic and squared relationship between data and target (because we generated the data).

Indeed, we could create two new features (data ** 2 and data ** 3) using this information as follows. This kind of transformation is called a polynomial feature expansion:

In [ ]:
data_expanded = np.concatenate([data, data ** 2, data ** 3], axis=1)
data_expanded.shape

A polynomial regression is nothing but a linear regression with polynomial features!

In [ ]:
linear_regression.fit(data_expanded, target)
target_predicted = linear_regression.predict(data_expanded)
mse = mean_squared_error(target, target_predicted)

In [ ]:
ax = sns.scatterplot(data=full_data, x="input_feature", y="target",
                     color="black", alpha=0.5)
ax.plot(data, target_predicted)
_ = ax.set_title(f"Mean squared error = {mse:.2f}")

We can see that even with a linear model, we can overcome the linearity limitation of the model by adding the non-linear components in the design of additional features. Here, we created new features by knowing the way the target was generated.

Instead of manually creating such polynomial features one could directly use [`sklearn.preprocessing.PolynomialFeatures`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html).

To demonstrate the use of the `PolynomialFeatures` class, we use a scikit-learn pipeline which first transforms the features and then fit the regression model.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

polynomial_regression = make_pipeline(
    PolynomialFeatures(degree=3, include_bias=False),
    LinearRegression(),
)
polynomial_regression.fit(data, target)
target_predicted = polynomial_regression.predict(data)
mse = mean_squared_error(target, target_predicted)

Generate a new feature matrix consisting of all polynomial combinations
of the features with degree less than or equal to the specified degree.
In this case, the `degree=3` polynomial features are [1, x, x^2, x^3].

In [ ]:
print(f"intercept: {polynomial_regression[-1].intercept_:.2f}, "
      f"weight_X: {polynomial_regression[-1].coef_[0]:.2f}, "
      f"weight_X^2: {polynomial_regression[-1].coef_[1]:.2f}, "
      f"weight_X^3: {polynomial_regression[-1].coef_[2]:.2f}, "
     )

We recall that `target` was created using `target = data ** 3 - 0.5 * data ** 2 + noise`.

In [ ]:
ax = sns.scatterplot(data=full_data, x="input_feature", y="target",
                     color="black", alpha=0.5)
ax.plot(data, target_predicted)
_ = ax.set_title(f"Mean squared error = {mse:.2f}")

As this procedure is equivalent to creating the features by hand (up to numerical error), it should not be surprising that the predictions of the `PolynomialFeatures` pipeline match the predictions of the linear model fit on manually engineered features.

## Cross-validation

Here we use cross-validation to estimate the stability of the weights found by the regression. First we get a visual intution of the `KFold` strategy.

In [ ]:
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

kf = KFold(n_splits=5, shuffle=True, random_state=0)
X_plot = np.linspace(-1.5,1.5,20)


i = 0
for train_index, test_index in kf.split(data):
    if i == 0:
        plt.scatter(full_data["input_feature"], full_data["target"],
        color="black", s=60,
        alpha=1, label="Original data")
        plt.title("Full data before cross-validation split", fontsize=14)
    i=i+1
    
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = target[train_index], target[test_index]
    
    polynomial_regression.fit(X_train, y_train)
    y_predicted = polynomial_regression.predict(X_plot.reshape(-1, 1))
    
    fig, ax = plt.subplots()
    ax.set_title(f"Fitted curve in Fold #{i} of KFold(shuffle=True)", fontsize=14)
    ax.scatter(X_test, y_test,
            color="tab:blue", facecolors="none",
            alpha=0.5, label="Resampled data", s=180, linewidth=5)
    ax.scatter(full_data["input_feature"], full_data["target"],
            color="black", s=60,
            alpha=1, label="Original data")


    ax.plot(X_plot, y_predicted, linewidth=2)

The `KFold` strategy will select `K` non-overlapping subsets of the data for testing. In this case we set `K=5` by passing the parameter `n_splits=5`, meaning that each fold will contain 20% of the data.

The `ShuffleSplit` strategy will draw subsets of possibly overlapping data for testing in each iteration. In this case the number of splits is independent of the `test_size` parameter.

In [ ]:
from sklearn.model_selection import cross_validate, ShuffleSplit

cv = ShuffleSplit(n_splits=40, test_size=0.1, random_state=0)

cv_results = cross_validate(polynomial_regression, data, target,
                            cv=cv, return_estimator=True,
                            scoring='neg_mean_squared_error')
cv_results = pd.DataFrame(cv_results)
cv_results["test_error"] = -cv_results["test_score"]

We get a different score in each fold. We have a **score distribution**.

In [ ]:
import matplotlib.pyplot as plt

cv_results["test_error"].plot.hist(bins=10, edgecolor="black", density=True)
plt.xlabel("Mean absolute error (k$)")
_ = plt.title("Test error distribution")

In [ ]:
print(f"The mean cross-validated testing error is: "
      f"{cv_results['test_error'].mean():.2f} +/-"
      f"{cv_results['test_error'].std():.2f}")

As we have different fits for different folds, we also have **weights distributions**.

In [ ]:
coefs = [pipeline[-1].intercept_ for pipeline in cv_results["estimator"]]
coefs = pd.DataFrame(coefs, columns=["intercept"])

coefs["weight_X"] = [pipeline[-1].coef_[0] for pipeline in cv_results["estimator"]]
coefs["weight_X^2"] = [pipeline[-1].coef_[1] for pipeline in cv_results["estimator"]]
coefs["weight_X^3"] = [pipeline[-1].coef_[2] for pipeline in cv_results["estimator"]]

print(f"intercept: {coefs['intercept'].mean():.2f} +/-"
      f"{coefs['intercept'].std():.2f}, "
      f"weight_X: {coefs['weight_X'].mean():.2f} +/-"
      f"{coefs['weight_X'].std():.2f}, "
      f"weight_X^2: {coefs['weight_X^2'].mean():.2f} +/-"
      f"{coefs['weight_X^2'].std():.2f}, "
      f"weight_X^3: {coefs['weight_X^3'].mean():.2f} +/-"
      f"{coefs['weight_X^3'].std():.2f}, "
     )

We could also use a histogram to visualize each coefficient, but a more intuitive way to display the set of equally valid fits is by using boxplots.

In [ ]:
ax0 = sns.boxplot(data=true_coefs, orient="h", color='red', medianprops=dict(color="red"))
ax1 = sns.boxplot(ax=ax0, data=coefs, orient="h", color='blue')


_ = plt.title("Linear regression coefficients")

In [ ]:
X_plot = np.linspace(-1.6, 1.6, 25)

for i in range(0,40):
    model = cv_results["estimator"][i]
    predictions = model.predict(X_plot.reshape(-1, 1))
    plt.plot(X_plot, predictions, linestyle="--", alpha=0.1,
             color="tab:blue")

plt.scatter(full_data["input_feature"], full_data["target"],
            color="black", s=60,
            alpha=1)

_ = plt.legend()

A confidence interval is the zone covered by the different sets of coefficients. Unfortunately, not all the scikit-learn regressions have implemented the option to provide the confidence interval, but tools such as `seaborn.regplot` do the job using bootstrap.

In [ ]:
fig, ax = plt.subplots()
fig.suptitle("Fitted curve with confidence interval", fontsize=16)
ax.set(xlim=(-1.5,1.5))
sns.regplot(data=full_data, x="input_feature", y="target", order=3,
           color="tab:blue", ax=ax)
ax = sns.scatterplot(data=full_data, x="input_feature", y="target", color="black", s=100)
#_ = plt.title("Bootstrap confidence interval")

From the boxplot we see that our model is not capturing the ground truth for the `intercept` nor the `weight_X`.

## Linear models with regularization

Examples of such models are [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html) and [`LassoCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html).

In [ ]:
from sklearn.linear_model import RidgeCV

alphas = np.logspace(-1, -0.2, num=50)

polynomial_regression = make_pipeline(
    PolynomialFeatures(degree=3, include_bias=False),
    RidgeCV(alphas=alphas, store_cv_values=True),
)

`RidgeCV` will automatically tune the regularization parameter using cross-validation, but one must keep an external, held-out test set for the final evaluation of the refitted model. We should use an additional cross-validation for this evaluation. This pattern is called nested cross-validation. In practice, we only need to embed the pipeline in the function `cross_validate` to perform such evaluation.

In [ ]:
cv_results = cross_validate(polynomial_regression, data, target,
                            cv=cv, scoring="neg_mean_squared_error",
                            return_estimator=True, n_jobs=2)
cv_results = pd.DataFrame(cv_results)
cv_results["test_error"] = -cv_results["test_score"]

In [ ]:
mse_alphas = [est[-1].cv_values_.mean(axis=0)
              for est in cv_results["estimator"]]
cv_alphas = pd.DataFrame(mse_alphas, columns=alphas)

In [ ]:
cv_alphas.mean(axis=0).plot(marker="+")
plt.ylabel("Mean squared error\n (lower is better)")
plt.xlabel("alpha")
_ = plt.title("Error obtained by cross-validation")

We found a sweetspot for the hyperparameter `alpha`. We can further question how stable is this sweetspot through the different trained models.

In [ ]:
best_alphas = [est[-1].alpha_ for est in cv_results["estimator"]]
print(f"The range of alphas leading to the best generalization performance is:\n"
      f"{np.mean(best_alphas):.2f} +/- {np.std(best_alphas):.2f}")

In [ ]:
print(f"The mean cross-validated testing error is: "
      f"{cv_results['test_error'].mean():.3f} +/-"
      f"{cv_results['test_error'].std():.3f}")

In [ ]:
coefs = [pipeline[-1].intercept_ for pipeline in cv_results["estimator"]]
coefs = pd.DataFrame(coefs, columns=["intercept"])

coefs["weight_X"] = [pipeline[-1].coef_[0] for pipeline in cv_results["estimator"]]
coefs["weight_X^2"] = [pipeline[-1].coef_[1] for pipeline in cv_results["estimator"]]
coefs["weight_X^3"] = [pipeline[-1].coef_[2] for pipeline in cv_results["estimator"]]

print(f"intercept: {coefs['intercept'].mean():.2f} +/-"
      f"{coefs['intercept'].std():.2f}, "
      f"weight_X: {coefs['weight_X'].mean():.2f} +/-"
      f"{coefs['weight_X'].std():.2f}, "
      f"weight_X^2: {coefs['weight_X^2'].mean():.2f} +/-"
      f"{coefs['weight_X^2'].std():.2f}, "
      f"weight_X^3: {coefs['weight_X^3'].mean():.2f} +/-"
      f"{coefs['weight_X^3'].std():.2f}, "
     )

In [ ]:
ax0 = sns.boxplot(data=true_coefs, orient="h", color='red', medianprops=dict(color="red"))
ax1 = sns.boxplot(ax=ax0, data=coefs, orient="h", color='blue')

_ = plt.title("Ridge regression coefficients")

Notice that `intercept` and `weight_X` are a bit shifted towards zero, but the `RidgeCV` model is still not capturing the ground truth for those coefficients. `LassoCV` uses a different kind of regularization that is able to set the weights of a fit exactly to zero.

In [ ]:
from sklearn.linear_model import LassoCV

polynomial_regression = make_pipeline(
    PolynomialFeatures(degree=3, include_bias=False),
    LassoCV(alphas=alphas, cv=cv)
)

In [ ]:
cv_results = cross_validate(polynomial_regression, data, target,
                            cv=cv, scoring="neg_mean_squared_error",
                            return_estimator=True, n_jobs=2)
cv_results = pd.DataFrame(cv_results)
cv_results["test_error"] = -cv_results["test_score"]

mse_alphas = [est[-1].alpha_ for est in cv_results["estimator"]]
cv_results['alphas'] = pd.DataFrame(mse_alphas)

cv_results.plot.scatter(x='alphas', y='test_error')

_ = plt.title("LassoCV best alphas")

In [ ]:
polynomial_regression[-1].set_params(alphas=cv_results['alphas'].unique())
polynomial_regression.fit(data, target)
target_predicted = polynomial_regression.predict(data)
mse = mean_squared_error(target, target_predicted)

In [ ]:
coefs = [pipeline[-1].intercept_ for pipeline in cv_results["estimator"]]
coefs = pd.DataFrame(coefs, columns=["intercept"])

coefs["weight_X"] = [pipeline[-1].coef_[0] for pipeline in cv_results["estimator"]]
coefs["weight_X^2"] = [pipeline[-1].coef_[1] for pipeline in cv_results["estimator"]]
coefs["weight_X^3"] = [pipeline[-1].coef_[2] for pipeline in cv_results["estimator"]]

print(f"intercept: {coefs['intercept'].mean():.2f} +/-"
      f"{coefs['intercept'].std():.2f}, "
      f"weight_X: {coefs['weight_X'].mean():.2f} +/-"
      f"{coefs['weight_X'].std():.2f}, "
      f"weight_X^2: {coefs['weight_X^2'].mean():.2f} +/-"
      f"{coefs['weight_X^2'].std():.2f}, "
      f"weight_X^3: {coefs['weight_X^3'].mean():.2f} +/-"
      f"{coefs['weight_X^3'].std():.2f}, "
     )

In [ ]:
ax0 = sns.boxplot(data=true_coefs, orient="h", color='red', medianprops=dict(color="red"))
ax1 = sns.boxplot(ax=ax0, data=coefs, orient="h", color='blue', medianprops=dict(alpha=0.5))

_ = plt.title("Lasso regression coefficients")

## Exercise

- Try other linear models with integrated parameter tuning such as [`ElasticNetCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNetCV.html). Alternatively, try Bayesian linear models such as [`BayesianRidge`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.BayesianRidge.html) or [`ARDRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ARDRegression.html)
- Try modifying the syntethic dataset: add noise, change its distribution, use linear combinations of trigonometric functions, exponentials, etc.